# 第十章 信号量和管程

## 10.1 背景

**并发问题**：竞态条件，多程序并发存在大的问题。

**同步**：
* 多线程共享公共数据的协调执行。
* 包括互斥与条件同步。

## 10.2 信号量

**信号量（Semaphore）**：一个抽象的数据类型，一个整形（sem），两种原子操作：
* P()：sem减1，如果sem < 0，进入等待，否则继续执行。
* V()：sem加1，如果sem <= 0，唤醒一个等待的P。

## 10.3 信号量的使用

**信号量的特性**：
* 信号量是整数。
* 信号量是被保护的变量，初始化完成后，之后P()和V()可以改变一个信号量的值；信号量的操作必须是原子的。
* P()能够阻塞，V()不会阻塞。

**两种信号量**：
* 二进制信号量：0或1。
* 一般/计数信号量：可以任何非负值。

**信号量的应用**：互斥和条件同步。

**二进制信号量用于实现互斥**

``` C++
mutex = new Semaphore(1);

// Thread
mutex->P();
    <critical section>
mutex->V();
```

**二进制信号量用于调度约束**

```
condition = new Semaphore(0);

// Thread A
...
condition->P();
    ...

// Thread B
     ...
condition->V();
...
```

## 10.4 信号量的实现


**信号量的实现**：使用了硬件的原语，禁用中断和test-and-set。

``` C++
class Semaphore {
    int sem;
    WaitQueue q;

    void P() {
        sem--;
        if (sem < 0) {
            <add this thread p to q>
            block(p);
        }
    }
    
    void V() {
        sem++;
        if (sem <= 0) {
            <Remove a thread t from q>
            wakeup(t);
        }
    }
};
```

## 10.5 管程

**管程（Monitor）目的**：分离互斥和条件同步的关注。

**定义**：一个用于指定临界区的锁；0或者多个条件变量（等待/通知信号量用于管理并发访问共享数据）

**管程的锁**：包含释放和抢占锁。

**管程中的条件变量**：
* 允许等待状态进入临界区：允许处于等待（睡眠）的线程进入临界区。
* Wait() 操作：释放锁，进入睡眠状态，重新获得锁后返回。
* Signal() 操作：唤醒所有等待者。

**实现**

``` C++
class Condition {
    int numWaiting = 0;
    WaitQueue q;
    
    void wait() {
        numWaiting++;
        <add this thread p to q>
        release(lock);
        schedule();
        require(lock);
    }
    
    void signal() {
        if (numWaiting > 0) {
            <remove a thread t from q>
            wakeup(t);
            numWaiting--;
        }
    }
};
```

## 10.6 读者-写者问题

读者-写者问题的动机是共享数据访问的问题，它有两种类型的使用者，读者（不需要修改数据）和写者（读取和修改数据）。

**问题的约束**：
* 允许同一时间有多个读者，但在任何时候只有一个写者。
* 当没有写者时，读者才能访问数据。
* 当没有读者和写者时，写者才能访问数据。
* 在任何时候只能有一个线程可以操作共享变量。

**共享数据**：数据集、信号量CountMutex（初始化为1）、信号量WriteMutex（初始化为1）、整数Rcount（初始化为0）。


**用信号量的方式实现读者优先的读者-写者问题**：只要有一个读者获取资源，后来读者都可以优先于写者获得资源。

``` C++
// Writer
sem_wait(WriteMutex);
    <write operation>
sem_post(WriteMutex);

// Reader
sem_wait(CountMutex);
if (Rcount == 0)
    sem_wait(WriteMutex);
++Rcount;
sem_post(CountMutex);

    <read operation>

sem_wait(CountMutex);
--Rcount
if (Rcount == 0)
    sem_post(WriteMutex);
sem_post(CountMutex);
```

**用管程的方式实现写者优先的读者-写者问题**：

``` C++
AR = 0; // Active Reader
AW = 0; // Active Writer
WR = 0; // Wait Reader
WW = 0; // Wait Writer
Condition okToRead;
Condition okToWrite;
Lock lock;

// Start Read
lock.Acquire();

while (AW + WW > 0) {
    WR++;
    okToRead.wait(&lock);
    WR--;
}
AR++;
lock.Release();

// Done Read
lock.Acquire();

AR--;
if (AR == 0 && WW > 0) {
    okToWrite.signal();
}

lock.Release();

// Start Write
lock.Acquire();

while (AR + AW > 0) {
    WW++;
    okToWrite.wait(&lock);
    WW--;
}
AW++;

lock.Release();

// Done Write
lock.Acquire();
AW--;
if (WW > 0) {
    okToWrite.signal();    // 只唤醒一个
} else if (WR > 0) {
    okToRead.broadcast();  // 唤醒全部
}
lock.Release();
```

## 10.7 哲学家就餐问题

**问题描述**：5个哲学家围绕一张圆桌，桌子上放着5支叉子，每两个哲学家之间放一支；哲学家的动作包含思考和进餐，进餐时需要同时拿起他左边和右边的两支叉子，思考时间则同时将两支叉子放回原处。

**共享数据**：数据集、信号量。

如果每个哲学家都先拿左边，再拿右边的叉子，可能会导致死锁。

**解法1**：要么不拿，要么就拿两把，哲学家拿叉子之前，会观察左右两个哲学家是否在进餐，都没有的话，才拿叉子进餐。

``` C++
# define N 5            // 哲学家个数
# define LEFT (i-1+N)%N         // 第i个哲学家的左邻居
# define RIGHT (i+1)%N  // 第i个哲学家的右邻居
# define THINKING 0     // 思考环节
# define HUNGRY         // 饥饿转置
# define EATING         // 进餐状态

int state[N];           // 记录每个人的状态

semaphore mutex;        // 互斥信号量，初值1
semaphore s[N];         // 同步信号量，初值0

// 哲学家
void philosopher(int i) {
    while(TRUE) {
        think();
        take_forks(i);
        eat();
        put_forks(i);
    }
}

void take_forks(int i) {
    P(mutex);
    state[i] = HUNGRY;   // 设置哲学家的状态
    test_take_left_right_forks(i);  // 试图拿两个叉子
    V(mutext);
    
    P(s[i]);   // 没有叉子的话便阻塞
}

void test_take_left_right_forks(int i) {
    if (state[i] == HUNGRY &&
         state[LEFT] != EATING &&
         state[RIGHT] != EATING) {
        state[i] = EATING;  // 两把叉子到手
        V(s[i]);    // 通知第i个人可以吃饭了
    }
}

void put_forks(int i) {
    P(mutex);
    state[i] = THINKING;
    test_take_left_right_forks(LEFT);
    test_take_left_right_forks(RIGHT);
    V(mutex);
}

```


**思路2**：奇数位的哲学家先拿左边的，偶数位的哲学家先拿右边的。